# FORNAX 3D Neutrino Spectra

Some I/O and plotting using the Fornax 3D models from [Vartanyan, Burrows, et al., MNRAS 482(1):351, 2019](https://arxiv.org/abs/1809.05106), which express the angular dependence of the neutrino luminosity from CCSNe in terms of a spherical harmonic expansion up to degree $\ell=2$. Note: Caching and plotting the angular dependence requires the `healpy` package, which can be installed via `pip install healpy`.

The data are available on [the Burrows group website](https://www.astro.princeton.edu/~burrows/nu-emissions.3d/).

In [ ]:
from snewpy.neutrino import Flavor
from snewpy.models.ccsn import Fornax_2019

from astropy import units as u

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.rc('font', size=16)
%matplotlib inline

## Initialization and Time Evolution of the Energy Spectrum

To start, let’s see what progenitors are available for the `Fornax_2019` model. We can use the `param` property to view all physics parameters and their possible values:

In [ ]:
Fornax_2019.param

We’ll initialise one of these progenitors and plot the evolution of the predicted neutrino spectrum across four time steps. If this is the first time you’re using a progenitor, snewpy will automatically download the required data files for you.

In [ ]:
model = Fornax_2019(progenitor_mass=10*u.solMass)
model

In [ ]:
E = np.arange(0,111) * u.MeV
theta = 23.55*u.degree
phi = 22.5*u.degree

fig, axes = plt.subplots(2,2, figsize=(11,8), sharex=True, sharey=True,
                         gridspec_kw = {'hspace':0.04, 'wspace':0.025})
axes = axes.flatten()

for ax, t in zip(axes, [10., 100., 300., 600.]):
    t *= u.ms
    spectra = model.get_initial_spectra(t, E, theta, phi)
    
    lines = ['--', '-.', '-', ':']
    for line, flavor in zip(lines, Flavor):
        ax.plot(E, spectra[flavor].to('1e52 erg/(s MeV)'), ls=line, label=flavor.to_tex())
    ax.legend(fontsize=14, ncol=2, loc='upper right', title='{:g}'.format(t))
    ax.set(xlim=(E[0].to_value('MeV'), E[-1].to_value('MeV')))
    ax.grid(ls=':')

axes[0].set(ylabel=r'$dL/dE$ [$10^{52}$ erg s$^{-1}$ MeV$^{-1}$]')
axes[2].set(xlabel=r'energy [MeV]');

fig.suptitle('Time Evolution of Spectra'.format(t))
fig.subplots_adjust(top=0.925, bottom=0.1)

## Test Caching of Full Model Angular Dependence

Here, if the user specifies `cache_flux=True` in the model constructor, the entire angular dependence of the model will be computed using a [HEALPix grid](https://healpy.readthedocs.io/en/latest/) with 192 pixels, using a HEALPix grid with parameter `nside=4` and `ordering=RING`. The model constructor will calculate the CCSN fluxes on the unit sphere at the centers of the 192 healpixels for all energy and time bins in the model.

The resulting tables are then saved to a FITS file in the same location as the input HDF5 file. If the user instantiates this model with `cache_flux=True` in the future, the model will be initialized using the cached FITS file.

Comments:
1. The HDF5 files store the angular dependence of the models in a highly compressed format using the spherical harmonic expansion up to degree $\ell=2$.
1. The time bins are equal across all flavors but the energy binning is a function of both time and flavor.
1. Computing the fluxes at 192 locations on the sky takes about 30 seconds per flavor due to the overhead of reading out the HDF5 files. So construction of the cached model is **slow** when called for the first time.
1. With `nside=4` and 192 bins, the angular resolution per bin is about $15^\circ$, which is reasonable given the highest-resolution features of the models are the quadrupole moments ($\ell=2$).

In [ ]:
cmodel = Fornax_2019(progenitor_mass=10*u.solMass, cache_flux=True)
cmodel

### Test Equivalence of the Cached and Uncached Models

In [ ]:
t = 100 * u.ms

Es, dEs, spectras = model._get_binnedspectra(t, theta, phi)
Ec, dEc, spectrac = cmodel._get_binnedspectra(t, theta, phi)

for flavor in Flavor:
    print('\n{}'.format(str(flavor)))
    print(Es[flavor])
    print(Ec[flavor])
    print(spectras[flavor])
    print(spectrac[flavor])
    
    j = np.argmax(spectrac[flavor])
    print('Max value:')
    print(Ec[flavor][j], spectrac[flavor][j])

In [ ]:
t = 600 * u.ms
E = np.arange(0,111) * u.MeV
theta = 23.55*u.degree
phi = 22.5*u.degree

spectra = model.get_initial_spectra(t, E, theta, phi)
cspectra = cmodel.get_initial_spectra(t, E, theta, phi)
    
fig, axes = plt.subplots(2,2, figsize=(11,8), sharex=True, sharey=True,
                         gridspec_kw = {'hspace':0.04, 'wspace':0.025})
axes = axes.flatten()

spectra = model.get_initial_spectra(t, E, theta, phi)

for ax, flavor in zip(axes, Flavor):
    ax.plot(E, spectra[flavor].to('1e52 erg/(s MeV)'), ls='-', lw=1, label='{}: model'.format(flavor.to_tex()))
    ax.plot(E, cspectra[flavor].to('1e52 erg/(s MeV)'), ls=':', lw=3, label='{}: cached model'.format(flavor.to_tex()))
    ax.legend(fontsize=14, ncol=1, loc='upper right')
    
    ax.set(xlim=(E[0].to_value('MeV'), E[-1].to_value('MeV')))
    ax.grid(ls=':')

axes[0].set(ylabel=r'$dL/dE$ [$10^{52}$ erg s$^{-1}$ MeV$^{-1}$]')
axes[2].set(xlabel=r'energy [MeV]')

fig.suptitle('Spectra at $t={{{:g}}}$'.format(t))
fig.subplots_adjust(top=0.925, bottom=0.1)

### Plot $L_\nu(t,\theta,\varphi)$ at Several Times

Plot the angular dependence $L(t)$ for several values of $t$.

In [ ]:
import healpy as hp

times = np.asarray([10., 100., 300., 600.]) * u.ms
nt = len(times)

fig, axes = plt.subplots(nt,3, figsize=(8*3, nt*4.5))

L_nue = cmodel.luminosity[Flavor.NU_E].to('1e52*erg/s')
L_nue_bar = cmodel.luminosity[Flavor.NU_E_BAR].to('1e52*erg/s')
L_nux = cmodel.luminosity[Flavor.NU_X].to('1e52*erg/s')

for i, t in enumerate(times):
    j = (np.abs(t - cmodel.time)).argmin()
    
    ax = axes[i,0]
    plt.axes(ax)
    hp.mollview(L_nue[j], title=r'$L_{{\nu_e}}(t={:.3f})$'.format(cmodel.time[j]),
                unit=r'$10^{52}$ erg s$^{-1}$', cmap='viridis',
                hold=True)
    
    ax = axes[i,1]
    plt.axes(ax)
    hp.mollview(L_nue_bar[j], title=r'$L_{{\bar{{\nu}}_e}}(t={:.3f})$'.format(cmodel.time[j]),
                unit=r'$10^{52}$ erg s$^{-1}$', cmap='magma',
                hold=True)
    
    ax = axes[i,2]
    plt.axes(ax)
    hp.mollview(L_nux[j], title=r'$L_{{\nu_X}}(t={:.3f})$'.format(cmodel.time[j]),
                unit=r'$10^{52}$ erg s$^{-1}$', cmap='cividis',
                hold=True)

### Plot $\Delta L_\nu(t,\theta,\varphi) / \langle L_\nu(t,\theta,\varphi)\rangle$ at Several Times

Same plot as above, but here plot the deviation from the average over the sky.

This demonstrates how the deviation from isotropy is very small initially and approaches 20% at later times.

In [ ]:
import healpy as hp

times = np.asarray([10., 100., 300., 600.]) * u.ms
nt = len(times)

fig, axes = plt.subplots(nt,3, figsize=(8*3, nt*4.5))

L_nue = cmodel.luminosity[Flavor.NU_E].to('1e52*erg/s')
L_nue_bar = cmodel.luminosity[Flavor.NU_E_BAR].to('1e52*erg/s')
L_nux = cmodel.luminosity[Flavor.NU_X].to('1e52*erg/s')

vmin, vmax = -0.2, 0.2

for i, t in enumerate(times):
    j = (np.abs(t - cmodel.time)).argmin()
    
    ax = axes[i,0]
    plt.axes(ax)
    Lavg = np.average(L_nue[j])
    dL_on_L = (L_nue[j] - Lavg) / Lavg
    hp.mollview(dL_on_L, title=r'$\nu_e$: $t=${:.3f}'.format(cmodel.time[j]),
                unit=r'$(L_\nu - \langle L_\nu\rangle) / \langle L_\nu\rangle$', cmap='viridis',
                min=vmin, max=vmax,
                hold=True)
    
    ax = axes[i,1]
    plt.axes(ax)
    Lavg = np.average(L_nue_bar[j])
    dL_on_L = (L_nue_bar[j] - Lavg) / Lavg
    hp.mollview(dL_on_L, title=r'$\bar{{\nu}}_e$: $t=${:.3f}'.format(cmodel.time[j]),
                unit=r'$\Delta L_{\bar{\nu}_e}/\langle L_{\bar{\nu}_e}\rangle$', cmap='magma',
                min=vmin, max=vmax,
                hold=True)
    
    ax = axes[i,2]
    plt.axes(ax)
    Lavg = np.average(L_nux[j])
    dL_on_L = (L_nux[j] - Lavg) / Lavg
    hp.mollview(dL_on_L, title=r'$\nu_X$: $t=${:.3f}'.format(cmodel.time[j]),
                unit=r'$(L_\nu - \langle L_\nu\rangle) / \langle L_\nu\rangle$', cmap='cividis',
                min=vmin, max=vmax,
                hold=True)

### Superimpose $L_\nu(t,\theta,\varphi)$ at All Locations on the Sphere

Plot $L(t)$ at all locations as well as the average, and then the deviation from the average.

#### Electron Neutrinos

In [ ]:
Lavg = np.average(L_nue, axis=1)
dL_over_L = (L_nue - Lavg[:,np.newaxis]) / Lavg[:,np.newaxis]

fig, axes = plt.subplots(2,1, figsize=(10,8),
                         gridspec_kw = {'height_ratios':[3,1], 'hspace':0},
                         sharex=True, tight_layout=True)

ax = axes[0]
ax.plot(cmodel.time, L_nue, color='gray', alpha=0.05)
ax.plot(cmodel.time, Lavg, color='r', ls='--')
ax.set(xlim=cmodel.time[0::len(cmodel.time)-1].value,
       ylabel=r'$L_{\nu_e}(t,\theta,\varphi)$ [$10^{52}$ erg s$^{-1}$]',
       ylim=(0, 1.1*np.max(L_nue).value),
       title=r'$L_{\nu_e}(t,\theta,\varphi)$')
ax.grid(ls=':')

ax = axes[1]
ax.plot(cmodel.time, dL_over_L, color='gray', alpha=0.1)
ax.plot(cmodel.time, np.zeros(cmodel.time.shape), ls='--', color='r')
ax.set(xlabel='time [s]',
       ylabel=r'$(L_\nu - \bar{L}_\nu)/\bar{L}_\nu$',
       ylim=(-0.3, 0.3))
ax.grid(ls=':');

#### Electron Antineutrinos

In [ ]:
Lavg = np.average(L_nue_bar, axis=1)
dL_over_L = (L_nue_bar - Lavg[:,np.newaxis]) / Lavg[:,np.newaxis]

fig, axes = plt.subplots(2,1, figsize=(10,8),
                         gridspec_kw = {'height_ratios':[3,1], 'hspace':0},
                         sharex=True, tight_layout=True)

ax = axes[0]
ax.plot(cmodel.time, L_nue_bar, color='gray', alpha=0.05)
ax.plot(cmodel.time, Lavg, color='r', ls='--')
ax.set(xlim=cmodel.time[0::len(cmodel.time)-1].value,
       ylabel=r'$L_{\bar{\nu}_e}(t,\theta,\varphi)$ [$10^{52}$ erg s$^{-1}$]',
       ylim=(0, 1.1*np.max(L_nue_bar).value),
       title=r'$L_{\bar{\nu}_e}(t,\theta,\varphi)$')
ax.grid(ls=':')

ax = axes[1]
ax.plot(cmodel.time, dL_over_L, color='gray', alpha=0.1)
ax.plot(cmodel.time, np.zeros(cmodel.time.shape), ls='--', color='r')
ax.set(xlabel='time [s]',
       ylabel=r'$(L_\bar{\nu} - \bar{L}_\bar{\nu})/\bar{L}_\bar{\nu}$',
       ylim=(-0.3, 0.3))
ax.grid(ls=':');

#### Flavor X (Stand-in for Mu and Tau Neutrinos)

Note that in this model, the mu and tau antineutrino flux is identical to the neutrino flux so we won't bother plotting it here.

In [ ]:
Lavg = np.average(L_nux, axis=1)
dL_over_L = (L_nux - Lavg[:,np.newaxis]) / Lavg[:,np.newaxis]

fig, axes = plt.subplots(2,1, figsize=(10,8),
                         gridspec_kw = {'height_ratios':[3,1], 'hspace':0},
                         sharex=True, tight_layout=True)

ax = axes[0]
ax.plot(cmodel.time, L_nux, color='gray', alpha=0.05)
ax.plot(cmodel.time, Lavg, color='r', ls='--')
ax.set(xlim=cmodel.time[0::len(cmodel.time)-1].value,
       ylabel=r'$L_{\nu_X}(t,\theta,\varphi)$ [$10^{52}$ erg s$^{-1}$]',
       ylim=(0, 1.1*np.max(L_nux).value),
       title=r'$L_{\nu_X}(t,\theta,\varphi)$')
ax.grid(ls=':')

ax = axes[1]
ax.plot(cmodel.time, dL_over_L, color='gray', alpha=0.1)
ax.plot(cmodel.time, np.zeros(cmodel.time.shape), ls='--', color='r')
ax.set(xlabel='time [s]',
       ylabel=r'$(L_\nu - \bar{L}_\nu)/\bar{L}_\nu$',
       ylim=(-0.3, 0.3))
ax.grid(ls=':');